# Handling Missing values 

In [8]:
from pyspark.sql import SparkSession
import pandas as pd

In [9]:
spark = SparkSession.builder.appName('spark-missingvalue-demao').getOrCreate()

In [18]:
df= pd.read_csv('test2.csv')
df.isnull().sum()

Name          2
age           1
Experience    2
Salary        1
dtype: int64

In [19]:
dfs = spark.read.csv('test2.csv',header=True)
dfs.show()

+---------+----+----------+------+
|     Name| age|Experience|Salary|
+---------+----+----------+------+
|    Krish|  31|        10| 30000|
|Sudhanshu|  30|         8| 25000|
|    Sunny|  29|         4| 20000|
|     Paul|  24|         3| 20000|
|   Harsha|  21|         1| 15000|
|  Shubham|  23|         2| 18000|
|   Mahesh|null|      null| 40000|
|     null|  34|        10| 38000|
|     null|  36|      null|  null|
+---------+----+----------+------+



## parameters of 'na.drop' function
how = 'any' --> drop row where any value is null 

how = 'all' --> drop row where all values are null

subset = ['columns_name'] --> removes/checks null only from that columns

thresh = number  --> minimum number of non-null values in a row

In [25]:
dfs.na.drop(how='any').show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [26]:
dfs.na.drop(how='any',subset=['Experience']).show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
|     null| 34|        10| 38000|
+---------+---+----------+------+



In [28]:
dfs.na.drop(how='any',thresh=3).show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
|     null| 34|        10| 38000|
+---------+---+----------+------+



## Filling missing values
fill(value,columns)

In [29]:
dfs.na.fill("Missing Value").show()

+-------------+-------------+-------------+-------------+
|         Name|          age|   Experience|       Salary|
+-------------+-------------+-------------+-------------+
|        Krish|           31|           10|        30000|
|    Sudhanshu|           30|            8|        25000|
|        Sunny|           29|            4|        20000|
|         Paul|           24|            3|        20000|
|       Harsha|           21|            1|        15000|
|      Shubham|           23|            2|        18000|
|       Mahesh|Missing Value|Missing Value|        40000|
|Missing Value|           34|           10|        38000|
|Missing Value|           36|Missing Value|Missing Value|
+-------------+-------------+-------------+-------------+



In [30]:
dfs.na.fill("Missing Value",['Experience','age']).show()

+---------+-------------+-------------+------+
|     Name|          age|   Experience|Salary|
+---------+-------------+-------------+------+
|    Krish|           31|           10| 30000|
|Sudhanshu|           30|            8| 25000|
|    Sunny|           29|            4| 20000|
|     Paul|           24|            3| 20000|
|   Harsha|           21|            1| 15000|
|  Shubham|           23|            2| 18000|
|   Mahesh|Missing Value|Missing Value| 40000|
|     null|           34|           10| 38000|
|     null|           36|Missing Value|  null|
+---------+-------------+-------------+------+



In [34]:
dfs.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- Experience: string (nullable = true)
 |-- Salary: string (nullable = true)



In [35]:
dfs = spark.read.csv('test2.csv',header=True,inferSchema=True)
dfs.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [36]:
from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols=['age', 'Experience', 'Salary'], 
    outputCols=["{}_imputed".format(c) for c in ['age', 'Experience', 'Salary']]
    ).setStrategy("mean")

In [37]:
# Add imputation cols to df
imputer.fit(dfs).transform(dfs).show()

+---------+----+----------+------+-----------+------------------+--------------+
|     Name| age|Experience|Salary|age_imputed|Experience_imputed|Salary_imputed|
+---------+----+----------+------+-----------+------------------+--------------+
|    Krish|  31|        10| 30000|         31|                10|         30000|
|Sudhanshu|  30|         8| 25000|         30|                 8|         25000|
|    Sunny|  29|         4| 20000|         29|                 4|         20000|
|     Paul|  24|         3| 20000|         24|                 3|         20000|
|   Harsha|  21|         1| 15000|         21|                 1|         15000|
|  Shubham|  23|         2| 18000|         23|                 2|         18000|
|   Mahesh|null|      null| 40000|         28|                 5|         40000|
|     null|  34|        10| 38000|         34|                10|         38000|
|     null|  36|      null|  null|         36|                 5|         25750|
+---------+----+----------+-

In [ ]:
##